In [ ]:
#"Hard" algorithm
#https://www.cs.princeton.edu/~chazelle/pubs/mstapprox.pdf 
#"Easy" algorithm
#http://www.wisdom.weizmann.ac.il/~oded/PTW/sublin.pdfs

In [ ]:
import networkx as nx
import random

n     = 2500
max_w = 50
c     = 10
eps   = 0.1

#G = nx.generators.random_graphs.random_regular_graph(3, n)

generator = nx.generators.cycle_graph
#generator = nx.generators.complete_graph

#n = n*c
#s.random_internet_as_graph(n)
G = generator(n)

for _ in range(c - 1):
    G_ = generator(n)
    G = nx.disjoint_union(G, G_)

n = n * c

for (u, v) in G.edges():
    G.edges[u,v]['weight'] = random.randint(1, max_w)

mst_gt = nx.algorithms.tree.mst.minimum_spanning_tree(G)

mst_gt_w = 0
#print(mst_gt.edges)
print(list(mst_gt.edges)[:10])
for (u,v) in mst_gt.edges:
    #print((u, v))
    mst_gt_w += G.edges[u,v]['weight']

print(mst_gt_w)


In [ ]:

import random
import sys
import time
from queue import Queue
from math import log10, log2

DEBUG = 0

RANDOM_SAMPLE_FACTOR = 6
SOME_BIG_CONSTANT    = 1500

#Vertex to [(neighbor, edge weight), ... ]
memos = {}
sg_memos = {}


def query_node_local(n):
    nbors = G.neighbors(n)
    return [(n2, G.edges[n, n2]["weight"]) for n2 in nbors]


def query_node_in_subgraph_n(n, w):
    if (n, w) not in memos:
        u_nbors = [v for v in query_node(n) if v[0] != n and v[1] <= w]
        u_nbors = list(set(u_nbors))
        memos[(n, w)] = u_nbors
        return memos[(n, w)]
    else:
        return memos[(n, w)]


def query_node(n):

    if DEBUG:
        return query_node_local(n)

    if n in memos:
        return memos[n]

    else:
        print(n)
        sys.stdout.flush()
        inp = input().strip("\n").split()[1:]
        memos[n] = []
        if len(inp) < 2:
            assert(len(inp) == 0)
            return memos[n]

        for i in range(0, len(inp), 2):
            #print("i", i, "len inp", len(inp), "inp", inp)
            memos[n].append((int(inp[i]), int(inp[i + 1])))
        
        return memos[n]

def approx_avg_degree(eps, C, n):
    import random
    n_ = 50 + int(C/eps)
    n = min(n_, 1+ n//1000)
    max_deg = 0

    for i in range(n_):
        node = random.randint(0, n - 1)
        out = query_node(node)

        deg = len(out)
        max_deg = max(max_deg, deg)

    return max_deg



def approx_cc_simple(n, gi, eps, max_w, d_bar, time_limit=None, start_time=None, read_new_verts=True, sample_size=0.5):

    #If eps > 1, still sample some constant or fraction
    r = 5*min(1000, log10(n)) + int(RANDOM_SAMPLE_FACTOR/eps**2) #RANDOM_SAMPLE_FACTOR)

    betas = 0
    
    i = 0
    betas = [0 for _ in range(max_w)]

    checkpoint_score_1 = 0
    checkpoint_score_2 = 1

    sampled = False

    while i < r:
        if (time.time() - start_time) > time_limit:
            break

        if i > 0 and i % r*sample_size == 0 and sampled == False:
            sampled = True
            r *= max(1, log2(betas[-1]))

        """
        if i > 0 and i % int(r * window) == 0 and int(r * window) > 0 and i > 500:
            if i == r*window:
                #First time
                #checkpoint_score_1 = [n / i * betas[j] for j in range(max_w)]
                pass
            else:
                checkpoint_score_2 = sum([n / i * betas[j] for j in range(max_w)])
                #if max(checkpoint_score_1 / checkpoint_score_2, checkpoint_score_2/checkpoint_score_1) < min(eps**2, 0.5**2)/100:
                if checkpoint_score_1 == checkpoint_score_2:
                    pass
                elif max(checkpoint_score_1 / checkpoint_score_2, checkpoint_score_2 / checkpoint_score_1) - 1 < min(eps**2, 0.5**2)/10:
                    break
                else:
                    pass
                checkpoint_score_1 = checkpoint_score_2
        """

        i += 1

        if read_new_verts:
            u = random.randint(0, n - 1)
        else:
            u = random.choice(list(memos.keys()))


        X = int(1/random.random())

        #betas[i] = 0
        for w in range(max_w, 0, -1):
            visited_nodes = set()
            visited_nodes.add(u)

            u_nbors_queue = Queue()
            u_nbors = [v for v in query_node(u) if v[0] != u and v[1] <= w]
            u_nbors = list(set(u_nbors))
            for nb in u_nbors:
                u_nbors_queue.put(nb)
            
            if DEBUG:
                for v in u_nbors:
                    assert(u_nbors.count(v) == 1)

            if u_nbors == []:
                #betas[i] = 1
                betas[w - 1] += 1
                continue

            j = -1
            heads = 0
            
            #Constant lookup for processed nodes
            added_to_queue = set([u])
            
            while not u_nbors_queue.empty():
                if (time.time() - start_time) > time_limit:
                    break

                #v = u_nbors.pop(0)
                v = u_nbors_queue.get()
                j += 1
                visited_nodes.add(v[0])
                v_nbors = query_node(v[0])
                v_nbors = sorted(v_nbors, key=lambda x: x[0] in memos, reverse=True)
                added = False
                for t in v_nbors:
                    if t[0] not in visited_nodes and t[0] not in added_to_queue and t[1] <= w:
                        u_nbors_queue.put(t)
                        added_to_queue.add(t[0])


                if j > X or len(visited_nodes) > X or len(added_to_queue) > X:
                    break            

                if u_nbors_queue.empty():
                    betas[w - 1] += 2**heads

                    toss = random.randint(0, 1)
                    if toss == 0:
                        break
                    else:
                        X *= 2
                        heads += 1

    if i <= 1:
        return [0 for _ in range(max_w)]

    return [n / i * betas[j] for j in range(max_w)]


def approx_mst(n, eps, max_w):

    d_bar = 0
    c_bars = []

    eps = eps / (1 + 0.5 * max_w)
    if max_w > 1:
        c_bars = approx_cc_simple(n, max_w, eps/2.5, max_w, d_bar, time_limit=2.9, start_time=time.time(), read_new_verts=True)
    else:
        c_bars = approx_cc_simple(n, max_w, eps/300, max_w, d_bar, time_limit=2.9, start_time=time.time(), read_new_verts=True)

    c_bars = [min(n, max(1, round(bar))) for bar in c_bars]
    c_bars[-1] = max(c_bars[-1], 1)

    if DEBUG == 1:
        print("est components", c_bars[-1])
        print("actual components", c)
        print("raw score", n  + sum(c_bars[1:]))

    return n + sum(c_bars[:-1]) - c_bars[-1] * max_w 


if not DEBUG:
    n = int(input())
    eps = float(input())
    max_w = int(input())


if n < 2:
    print("end " + str(0))

if max_w < 1:
    print("end " + str(0))


#Group 6 has max_w > 50
#if max_w > 50:
#    1/0


mst = approx_mst(n, eps, max_w)
mst = max(n/2, mst)
mst = min((n - 1) * max_w, mst)

print("end " + str(mst))
sys.stdout.flush()


if DEBUG == 1:
    print("gt end", mst_gt_w)

In [ ]:
#Hard version
import random
import sys

DEBUG = 1

RANDOM_SAMPLE_FACTOR = 200
SOME_BIG_CONSTANT    = 10

#Vertex to [(neighbor, edge weight), ... ]
memos = {}

def query_node_local(n):
    nbors = G.neighbors(n)
    return [(n2, G.edges[n, n2]["weight"]) for n2 in nbors]

def query_node(n):

    if DEBUG:
        return query_node_local(n)

    if n in memos:
        return memos[n]

    else:
        print(n)
        sys.stdout.flush()
        inp = input().strip("\n").split()[1:]
        memos[n] = []
        if len(inp) < 2:
            return memos[n]

        for i in range(0, len(inp), 2):
            #print("i", i, "len inp", len(inp), "inp", inp)
            memos[n].append((int(inp[i]), int(inp[i + 1])))
        
        return memos[n]

def approx_avg_degree(eps, C, n):
    import random
    n_ = 50 + int(C/eps)
    n = min(n_, 1+ n//1000)
    max_deg = 0

    for i in range(n_):
        node = random.randint(0, n - 1)
        out = query_node(node)

        deg = len(out)
        max_deg = max(max_deg, deg)


    return max_deg


def approx_cc(n, gi, eps, max_w, d_bar):
    import random
    import sys

    r = int(1/eps**2) * RANDOM_SAMPLE_FACTOR


    #Does not check dupes
    vs = [random.randint(0, n - 1) for _ in range(r)]
    #Remove dupes
    #for v in vs:
    #    for _ in range(vs.count(v) - 1):
    #        vs.remove(v)
    

    r = len(vs)
    betas = [0 for _ in range(int(r))]


    for i, u in enumerate(vs):

        # Start a BFS that exits probabilistically 

        seen_edges      = set()
        seen_nodes      = set()
        visited_nodes   = set()
        visited_edges   = set()
        n_seen_edges    = 0      #Seen and visited are the same
        n_seen_nodes    = 0
        n_visited_nodes = 0
        n_visited_edges = 0
        max_degree_seen = 0

        u_neighs = query_node(u)
        u_neighs = [t for t in u_neighs if t[1] <= gi]

        singleton = False

        if len(u_neighs) == 0:
            singleton = True
            betas[i] = 2
            continue

        throws = 0

        dui = len(u_neighs)
        walk_limit = len(u_neighs)
        
        max_degree_seen = len(u_neighs)

        edges_by_depth = [[((u, v[0]), v[1])  for v in u_neighs]]

        j = -1
        while len(edges_by_depth) > 0:
            j += 1
            if DEBUG == 1:
                pass
                #print("start/finish walk, our search tree looks like")
                #print(edges_by_depth)
            u_edges = edges_by_depth.pop(0)
            #edge :: tuple (tuple (int u, int v), float weight)

            #Take a step
            edges_by_depth.append([])
            #V is an edge
            if DEBUG == 1:
                pass
                #print("start depth", j)
            for v in u_edges:
                if v[0][1] not in visited_nodes:
                    visited_nodes.add(v[0][1])
                    n_visited_nodes += 1
                    visited_edges.add(v[0][1])
                    n_visited_edges += 1

                ts = query_node(v[0][1])
                max_degree_seen = max(max_degree_seen, len(ts))

                #ts is a list of tuples (vertex, weight)
                for t in ts:
                    if ((v[0][1], t[0]), t[1]) not in seen_edges:
                        seen_edges.add(((v[0][1], t[0]), t[1]))
                        n_seen_edges += 1
                    if t[0] not in visited_nodes:
                        visited_nodes.add(t[0])
                        n_visited_nodes += 1
                        edges_by_depth[-1].append(((v[0][0], t[0]), t[1]))

            if edges_by_depth[-1] == []:
                edges_by_depth.pop()


            if edges_by_depth.count([]) == len(edges_by_depth):
                #Finished bfs
                betas[i] = dui * 2**(throws + 1)

            if max_degree_seen > d_bar or len(visited_nodes) > max_w:
                break

            if len(seen_edges) < walk_limit:
                toss = random.randint(0, 1)
                throws += 1
                if toss == 1:
                    walk_limit = walk_limit*2
                else:
                    break
                #max_degree_seen = max(max_degree_seen, dui)
   
    return n / (2 * r) * sum(betas)


def simple(n, d, eps, max_w):
    sum_ = 0
    s = int(4 / eps**2)
    for i in range(s):
        seen_nodes = set()
        queue_nodes = [random.randint(0, n-1)]
        while len(seen_nodes) <= 2/eps and len(queue_nodes) > 0:
            #bfs
            u = queue_nodes.pop(0)
            seen_nodes.add(u)
            nbors = query_node(u)
            for b in nbors:
                if b[0] not in seen_nodes and b[1] <= max_w:
                    queue_nodes.append(b[0])
                    seen_nodes.add(b[0])

        #print("seen nodes", seen_nodes, "queue_nodes", queue_nodes, "2/eps", 2/eps)
        if len(seen_nodes) > 2/eps:
            sum_ += 1./len(seen_nodes)
        elif len(seen_nodes) <= 1:
            sum_ += 1
        else:
            sum_ += eps/2.

    return n * sum_ / s
            
            





def approx_mst(n, eps, max_w):

    d_bar = approx_avg_degree(eps, SOME_BIG_CONSTANT, n)
    #d_bar = 0
    
    c_bars = []
    for i in range(1, max_w):

        #cc_bar = approx_cc(n, i , eps/2, 4*max_w/eps, d_bar)
        cc_bar = simple(n, d_bar, eps, i)   
        c_bars.append(cc_bar)

    #all_ = approx_cc(n, max_w, 0.01, 4*max_w/eps, d_bar)
    all_ = simple(n, d_bar, eps, max_w)
    if DEBUG == 1:
        print("gt c", c)
        print("est c", all_)
        print("raw est", n - max_w + sum(c_bars))
        print("approx avg (max?) degree", d_bar)

    return n - max_w + sum(c_bars) - d_bar

if not DEBUG:
    n = int(input())
    eps = float(input())
    max_w = int(input())

assert (max_w >= 1)

#print(max_w)

mst = approx_mst(n, eps, max_w)

#assert(mst >= n/2)

print("end " + str(mst))
sys.stdout.flush()

if DEBUG == 1:
    print("gt end", mst_gt_w)

